<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# **Introduction to Meridian Demo**

Welcome to the Meridian end-to-end demo. This simplified demo showcases the fundamental functionalities and basic usage of the library, including working examples of the major modeling steps:


<ol start="0">
  <li><a href="#install">Install</a></li>
  <li><a href="#load-data">Load the data</a></li>
  <li><a href="#configure-model">Configure the model</a></li>
  <li><a href="#model-diagnostics">Run model diagnostics</a></li>
  <li><a href="#generate-summary">Generate model results & two-page output</a></li>
  <li><a href="#generate-optimize">Run budget optimization & two-page output</a></li>
  <li><a href="#save-model">Save the model object</a></li>
</ol>


Note that this notebook skips all of the exploratory data analysis and preprocessing steps. It assumes that you have completed these tasks before reaching this point in the demo.

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.

<a name="install"></a>
## Step 0: Install

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Install the latest version of Meridian, and verify that GPU is available.

In [2]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.1.1

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git@main"

In [3]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

2025-07-28 11:26:27.124918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753701987.158120   34091 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753701987.166387   34091 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 11:26:27.185091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Your runtime has 8.3 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


2025-07-28 11:26:31.194890: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<a name="load-data"></a>
## Step 1: Load the data

Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/geo_all_channels.csv) as follows.

1\. Read the data into a Pandas DataFrame.

In [4]:
df = pd.read_csv("maudau 3.csv")

df.head()

types={"алкоголь","детские","зоотовары","красота","продукты_питания", "дома"}
main=pd.DataFrame()
raw_time=['2021-01-25', '2021-02-01', '2021-02-08', '2021-02-15',
       '2021-02-22', '2021-03-01', '2021-03-08', '2021-03-15',
       '2021-03-22', '2021-03-29', '2021-04-05', '2021-04-12',
       '2021-04-19', '2021-04-26', '2021-05-03', '2021-05-10',
       '2021-05-17', '2021-05-24', '2021-05-31', '2021-06-07',
       '2021-06-14', '2021-06-21', '2021-06-28', '2021-07-05',
       '2021-07-12', '2021-07-19', '2021-07-26', '2021-08-02',
       '2021-08-09', '2021-08-16', '2021-08-23', '2021-08-30',
       '2021-09-06', '2021-09-13', '2021-09-20', '2021-09-27',
       '2021-10-04', '2021-10-11', '2021-10-18', '2021-10-25',
       '2021-11-01', '2021-11-08', '2021-11-15', '2021-11-22',
       '2021-11-29', '2021-12-06', '2021-12-13', '2021-12-20',
       '2021-12-27', '2022-01-03', '2022-01-10', '2022-01-17',
       '2022-01-24', '2022-01-31', '2022-02-07', '2022-02-14',
       '2022-02-21', '2022-02-28', '2022-03-07', '2022-03-14',
       '2022-03-21', '2022-03-28', '2022-04-04', '2022-04-11',
       '2022-04-18', '2022-04-25', '2022-05-02', '2022-05-09',
       '2022-05-16', '2022-05-23', '2022-05-30', '2022-06-06',
       '2022-06-13', '2022-06-20', '2022-06-27', '2022-07-04',
       '2022-07-11', '2022-07-18', '2022-07-25', '2022-08-01',
       '2022-08-08', '2022-08-15', '2022-08-22', '2022-08-29',
       '2022-09-05', '2022-09-12', '2022-09-19', '2022-09-26',
       '2022-10-03', '2022-10-10', '2022-10-17', '2022-10-24',
       '2022-10-31', '2022-11-07', '2022-11-14', '2022-11-21',
       '2022-11-28', '2022-12-05', '2022-12-12', '2022-12-19',
       '2022-12-26', '2023-01-02', '2023-01-09', '2023-01-16',
       '2023-01-23', '2023-01-30', '2023-02-06', '2023-02-13',
       '2023-02-20', '2023-02-27', '2023-03-06', '2023-03-13',
       '2023-03-20', '2023-03-27', '2023-04-03', '2023-04-10',
       '2023-04-17', '2023-04-24', '2023-05-01', '2023-05-08',
       '2023-05-15', '2023-05-22', '2023-05-29', '2023-06-05',
       '2023-06-12', '2023-06-19', '2023-06-26', '2023-07-03',
       '2023-07-10', '2023-07-17', '2023-07-24', '2023-07-31',
       '2023-08-07', '2023-08-14', '2023-08-21', '2023-08-28',
       '2023-09-04', '2023-09-11', '2023-09-18', '2023-09-25',
       '2023-10-02', '2023-10-09', '2023-10-16', '2023-10-23',
       '2023-10-30', '2023-11-06', '2023-11-13', '2023-11-20',
       '2023-11-27', '2023-12-04', '2023-12-11', '2023-12-18',
       '2023-12-25', '2024-01-01', '2024-01-08', '2024-01-15']
main["time"]=raw_time[0:62]
i=0
for typ in types:
    sorted_df=df[df['Google Ads campaign'].str.contains(typ, case=False, na=False)].groupby('Nth week').sum(numeric_only=True).reset_index()
    main[f"Channel{i}_impression"]=sorted_df["Google Ads impressions"]
    main[f"Channel{i}_spend"]=sorted_df["Google Ads cost"]
    i+=1
main["kpi"]=df.groupby('Nth week').sum(numeric_only=True).reset_index()["Active users"]
main.tail()



,time,Channel0_impression,Channel0_spend,Channel1_impression,Channel1_spend,Channel2_impression,Channel2_spend,Channel3_impression,Channel3_spend,Channel4_impression,Channel4_spend,Channel5_impression,Channel5_spend,kpi
57,2022-02-28,1473412,244880.850608,1754597,178147.799410,2787210,329970.076982,4059915,323694.537582,4250925,437596.655935,2874870,297917.042791,68350
58,2022-03-07,1426894,221030.067248,1826641,202761.717035,2638793,321478.361031,4067663,325275.820388,4657204,465746.149801,2835056,292233.941942,68193
59,2022-03-14,1279515,179006.764816,1823126,196001.491278,2729023,314894.357215,4102369,327970.994681,4582528,449431.113501,2337487,228765.569779,68093
60,2022-03-21,1209290,186582.970792,1713195,178460.576866,2597880,284421.468368,4291079,323971.871878,4542763,454370.544621,2632177,263299.782832,69225
61,2022-03-28,146524,24156.420754,267633,26892.862054,370998,41760.169421,649884,48909.160749,649293,60276.761124,388184,39232.003622,10847


2\. Create a DataFrameInputDataBuilder instance.

In [5]:
builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
)

3\. Offer the components to the builder. Note that the components may be offered all at once or piecewise.

In [6]:
builder = (
    builder.with_kpi(main)
)

channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4", "Channel5"]
builder = builder.with_media(
    main,
    media_cols=[f"{channel}_impression" for channel in channels],
    media_spend_cols=[f"{channel}_spend" for channel in channels],
    media_channels=channels,
)

data = builder.build()

/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/data/input_data.py:471: UserWarning: Consider setting custom priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(


Note that the simulated data here does not contain reach and frequency. We recommend including reach and frequency data whenever they are available. For information about the advantages of utilizing reach and frequency, see [Bayesian Hierarchical Media Mix Model Incorporating Reach and Frequency Data](https://research.google/pubs/bayesian-hierarchical-media-mix-model-incorporating-reach-and-frequency-data/#:~:text=By%20incorporating%20R%26F%20into%20MMM,based%20on%20optimal%20frequency%20recommendations.). For code snippet for loading reach and frequency data, see [Load geo-level data with reach and frequency](https://developers.google.com/meridian/docs/user-guide/load-geo-data-with-rf)

The documentation provides guidance for instances where reach and frequency data is accessible for specific channels. Additionally, for information about how to load other data types and formats, including data with reach and frequency, see [Supported data types and formats](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats).

<a name="configure-model"></a>
## Step 2: Configure the model

Meridian uses Bayesian framework and Markov Chain Monte Carlo (MCMC) algorithms to sample from the posterior distribution.

1\. Inititalize the `Meridian` class by passing the loaded data and the customized model specification. One advantage of Meridian lies in its capacity to calibrate the model directly through ROI priors, as described in [Media Mix Model Calibration With Bayesian Priors](https://research.google/pubs/media-mix-model-calibration-with-bayesian-priors/). In this particular example, the ROI priors for all media channels are identical, with each being represented as Lognormal(0.2, 0.9).

In [7]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/model.py:66: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/model.py:867: UserWarning: Consider setting custom ROI priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at  https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(
I0000 00:00:1753701991.577576   34091 service.cc:148] XLA service 0x6328f3ea1220 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753701991.577645   34091 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1753701991.589760   34091 dev

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.

In [ ]:
import multiprocessing

num_cores = multiprocessing.cpu_count()

def worker_function(x):
    return x * x

if __name__ == "__main__":
    num_cores = multiprocessing.cpu_count()

    # Create a pool of workers equal to the number of cores
    with multiprocessing.Pool(num_cores) as pool:
        results = pool.map(worker_function, range(10))

    print(results)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


: 

In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/meridian/model/prior_distribution.py:915: UserWarning: Hierarchical distribution parameters must be deterministically zero f

For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).

<a name="model-diagnostics"></a>
## Step 3: Run model diagnostics

After the model is built, you must assess convergence, debug the model if needed, and then assess the model fit.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

2\. Assess the model's fit by comparing the expected sales against the actual sales.

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).

<a name="generate-summary"></a>
## Step 4: Generate model results & two-page output

To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive'
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary(
    'summary_output.html', filepath, start_date, end_date
)

Here is a preview of the two-page output based on the simulated data:

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/summary_output.html')

For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).





<a name="generate-optimize"></a>
## Step 5: Run budget optimization & generate an optimization report

You can choose what scenario to run for the budget allocation. In default scenario, you find the optimal allocation across channels for a given budget to maximize the return on investment (ROI).

1\. Instantiate the `BudgetOptimizer` class and run the `optimize()` method without any customization, to run the default library's Fixed Budget Scenario to maximize ROI.

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

2\. Export the 2-page HTML optimization report, which contains optimized spend allocations and ROI.

In [ ]:
filepath = '/content/drive/MyDrive'
optimization_results.output_optimization_summary(
    'optimization_output.html', filepath
)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/optimization_output.html')

For information about customized optimization scenarios, such as flexible budget scenarios, see [Budget optimization scenarios](https://developers.google.com/meridian/docs/user-guide/budget-optimization-scenarios). For more information about optimization results summary and individual visualizations, see [optimization results output](https://developers.google.com/meridian/docs/user-guide/generate-optimization-results-output) and [optimization visualizations](https://developers.google.com/meridian/docs/user-guide/plot-optimization-visualizations).

<a name="save-model"></a>
## Step 6: Save the model object

We recommend that you save the model object for future use. This helps you to  avoid repetitive model runs and saves time and computational resources. After the model object is saved, you can load it at a later stage to continue the analysis or visualizations without having to re-run the model.


Run the following codes to save the model object:

In [ ]:
file_path = '/content/drive/MyDrive/saved_mmm.pkl'
model.save_mmm(mmm, file_path)

Run the following codes to load the saved model:

In [ ]:
mmm = model.load_mmm(file_path)